In [1]:
from Supporting_functions import *
from WBNS import WBNS_image
import RedLionfishDeconv as rl

import scipy.ndimage as ndi
from aicsimageio.readers import CziReader
from aicsimageio.writers import OmeTiffWriter
import re

from skimage import measure  

import os
import numpy as np
import tifffile
from tqdm import tqdm

/home/hmorales/miniconda3/envs/selfnet/lib/python3.9/site-packages/pydantic/_migration.py:281: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [2]:
# Input



## Tiff source folders

# Source paths for the tiff images if extractImagesFromCzi== True, the folders will be created
srcpath = r'/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/'
dirOut    = r'/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/SqtsfGFP_test/'

dirSource = srcpath + 'SqtsfGFP_allData/'  # this folder contain the tiffs


## Generate image planes for training data 
dirTarget = dirSource

## Process Target image 
processTargetImages = True

# BG subtraction: not necessary
resolution_px = 0 # FWHM of the PSF : 2 to 5 (~ radius of the structure , > 1)
resolution_pz = 0
noise_lvl = 2

# deconvolution: not necessary
psf_path = r'/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/Averaged_transformed_PSF_488.tif'
padding = 32
Niter = 0 # No pre-2D deconvolution

# post processing
sigmaLoG = 0.0
sigmaLoGAddScale = 0.0
sigma = 0.8

# image normalization : Better no change
thres_scale_target = 2.0 # threhold for simple mask for normalization
percentiles_target = (10, 99.9999)
min_v_target = 0
max_v_target = 65535
# save processed images
dirProcTarget = srcpath + 'SqtsfGFP_allData_processed/' 

## Process Source image
# image normalization
processSourceImages = True
thres_scale_source = 1.5  # threhold for simple mask for normalization
percentiles_source = (10, 99.9999)
min_v_source = 0
max_v_source = 65535



## Parameters for the patch selection
patch_size = 64
stride = 64
signal_intensity_threshold = 10000  #parameter for selecting image patches containing signals
add_flipped_pathes = True

xy_interval=2
xz_interval=2

# Generate image planes for training data 

In [3]:
## Create folders to export data
outDir = dirOut+"raw_data/"
xy_data = outDir+"xy/"
xy_lr_data = outDir+"xy_lr/"
xz_data = outDir+"xz/"
   
createFolder(dirOut)
createFolder(outDir)
createFolder(xy_data)
createFolder(xy_lr_data)
createFolder(xz_data)

folder: '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/SqtsfGFP_test/' already exists
folder: '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/SqtsfGFP_test/raw_data/' already exists
folder: '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/SqtsfGFP_test/raw_data/xy/' already exists
folder: '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/SqtsfGFP_test/raw_data/xy_lr/' already exists
folder: '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/SqtsfGFP_dataforHernan/croppedfortraining/Models/SqtsfGFP_test/raw_data/xz/' already exists


## Generate image planes for target and intermediate images

In [4]:
# Get all tif images in the target folder, process and export them
image_names = sorted([f for f in os.listdir(dirTarget) if f.endswith('.tif')])

if processTargetImages == True:
    #Creat output folder
    createFolder(dirProcTarget)
    # Open PSF and Prepare PSF
    psf = tifffile.imread(psf_path)
    psf_f = psf.astype(np.float32)
    psf_norm = psf_f/psf_f.sum()

    
count = 1
for i, image_name in tqdm(enumerate(image_names)):

    start_time = time.time()  # Record the start time 
    print(f"** Processing image : {image_name}")
    
    # Open image and get metadata
    img_path = os.path.join(dirTarget, image_name)   
    img = tifffile.imread(img_path)
    img_shape = img.shape
    [physical_pixel_sizeX,physical_pixel_sizeY,physical_pixel_sizeZ] = read_tiff_voxel_size(img_path)    
    scale = physical_pixel_sizeX / physical_pixel_sizeZ
    

    
    if processTargetImages == True:
        # Make image isotropic
        if abs(1.0-scale) > 1e-4: 
            img = reslice(img,'xy',physical_pixel_sizeX,physical_pixel_sizeZ)
        img = img.astype(np.float32)
        new_img_shape = img.shape   
        new_physical_pixel_sizeZ = img_shape[0] * physical_pixel_sizeZ / new_img_shape[0]
        print(f"     -image dimension from : {img_shape} to {new_img_shape}")
        print(f"     -z-space from : {physical_pixel_sizeZ} to {new_physical_pixel_sizeZ}")

        # Get mask
        mask = get_image_simple_mask(img, 0.0, 1.0, thres_scale_target)  
        mask =  mask.astype(np.int16)
    
        # Deconvolution
        if Niter > 0: 
            # Padding image
            img = np.pad(img, padding, mode='reflect')
            imgSizeGB = img.nbytes / (1024 ** 3)
            print('     -size(GB) : ', imgSizeGB)
            # GPU deconvolution
            res_gpu = rl.doRLDeconvolutionFromNpArrays(img, psf, niter=Niter,resAsUint8=False)
            # Removing padding
            img = res_gpu[padding:-padding, padding:-padding, padding:-padding]

        # Remove noise and BG
        if resolution_px > 0:
            img = WBNS_image(img, resolution_px, noise_lvl)
            if resolution_pz > 0:
                img_xz=np.transpose(img,[1,0,2])
                img_xz = WBNS_image(img_xz, resolution_pz, 0)
                img = np.transpose(img_xz,[1,0,2])
        # LoG filter
        if sigmaLoG > 0 :
            imgBorders = ndi.gaussian_laplace(img, sigmaLoG)
            imgBorders *= -1.0
            #imgBorders[imgBorders < 0] = 0
            imgBorders *= sigmaLoGAddScale
            img += imgBorders
        
        # Smooth
        if sigma > 0:
            img = ndi.gaussian_filter(img, sigma)
        
        # Image Normalization
        if percentiles_target[0] > 0 or percentiles_target[1] < 100:
            low_thres, high_thres0 = getNormalizationThresholds(img, percentiles_target) # low thres in whole image
            low_thres0, high_thres = getNormalizationThresholds(img * mask, percentiles_target) # high thres in FG
            img = remove_outliers_image(img, low_thres, high_thres)

        img = image_scaling(img, min_v_target, max_v_target, True)
        img = img.astype(np.uint16)
        
        # Save processed image
        img_out_name = os.path.join(dirProcTarget, image_name)            
        tifffile.imwrite(      
            img_out_name,
            img,
            imagej=True, 
            resolution=(1.0/physical_pixel_sizeX, 1.0/physical_pixel_sizeY), 
            metadata={'spacing': new_physical_pixel_sizeZ, 'unit': 'um', 'axes': 'ZYX'})
        
    # Generate intermediate images down sample and then upsample
    img_lr = np.zeros_like(img)
    z,y,x = img.shape    
    new_y = round(y * scale)
    new_x = round(x * scale)
    
    for i in range(z):
        temp_img  = cv2.resize(img[i,:,:],(new_x,new_y),interpolation=cv2.INTER_CUBIC)
        #img_lr[i,:,:] = cv2.resize(temp_img,(x,y),interpolation=cv2.INTER_CUBIC)  
        img_lr[i,:,:] = cv2.resize(temp_img,(x,y),interpolation=cv2.INTER_LINEAR) 
            
    # Export planes for target and intermediate images, each plane as a TIFF image 
    
    for i in range(z):
        outName_target = f"{xy_data}{count}.tif"
        outName_interm = f"{xy_lr_data}{count}.tif"
        tifffile.imwrite(outName_target, img[i,:,:])
        tifffile.imwrite(outName_interm, img_lr[i,:,:])      
        count += 1
  
    Elapsed_time = time.time() - start_time
    print(f"Elapsed Time: {Elapsed_time:.4f} seconds, image {image_name}, {count-1} images exported ") 

0it [00:00, ?it/s]

** Processing image : 18degr_singleTrack__2023_12_18__15_49_26_471(190)crop.tif
     -image dimension from : (26, 321, 399) to (84, 321, 399)
     -z-space from : 1.8695295485696815 to 0.5786639078906157
     -threshold_value: 434.0
Intensity Norm  from (0 , 778) to  (0, 65535)  



1it [00:02,  2.87s/it]

Elapsed Time: 2.8695 seconds, image 18degr_singleTrack__2023_12_18__15_49_26_471(190)crop.tif, 84 images exported 
** Processing image : 18degr_singleTrack__2023_12_18__15_49_26_471(200)crop.tif
     -image dimension from : (51, 423, 420) to (165, 423, 420)
     -z-space from : 1.8695295485696815 to 0.5778545877397196
     -threshold_value: 424.0
Intensity Norm  from (0 , 918) to  (0, 65534)  



2it [00:09,  4.75s/it]

Elapsed Time: 6.6275 seconds, image 18degr_singleTrack__2023_12_18__15_49_26_471(200)crop.tif, 249 images exported 


## Generate image planes for source images

In [5]:
# Get all tif images in the dirSource folder, process and export them
image_names = sorted([f for f in os.listdir(dirSource) if f.endswith('.tif')])

count = 1
for i, image_name in tqdm(enumerate(image_names)):

    start_time = time.time()  # Record the start time 
    print(f"** Processing image : {image_name}")
    
    # Open image and get metadata
    img_path = os.path.join(dirSource, image_name)   
    img = tifffile.imread(img_path)
    img_shape = img.shape
    [physical_pixel_sizeX,physical_pixel_sizeY,physical_pixel_sizeZ] = read_tiff_voxel_size(img_path)    
    scale = physical_pixel_sizeX / physical_pixel_sizeZ
 

    if processSourceImages: 
        # Get mask
        mask = get_image_simple_mask(img, 0.0, 1.0, thres_scale_source)  
        mask =  mask.astype(np.int16)
        # Image Normalization
        if percentiles_source[0] > 0 or percentiles_source[1] < 100:
            low_thres, high_thres0 = getNormalizationThresholds(img, percentiles_source) # low thres in whole image
            low_thres0, high_thres = getNormalizationThresholds(img * mask, percentiles_source) # high thres in FG
            img = remove_outliers_image(img, low_thres, high_thres)

        img = image_scaling(img, min_v_source, max_v_source, True)
        img = img.astype(np.uint16)

    # reslice Image
    img_xz = reslice(img,'xz',physical_pixel_sizeX,physical_pixel_sizeZ)
    
    z,y,x = img_xz.shape 
    
    for i in range(z):
        outName_target = f"{xz_data}{count}.tif"
        tifffile.imwrite(outName_target, img_xz[i,:,:])
        count += 1
  
    Elapsed_time = time.time() - start_time
    print(f"Elapsed Time: {Elapsed_time:.4f} seconds, image {image_name}, {count-1} images exported ")    
    

0it [00:00, ?it/s]

** Processing image : 18degr_singleTrack__2023_12_18__15_49_26_471(190)crop.tif
     -threshold_value: 324.0
Intensity Norm  from (0 , 1041) to  (0, 65535)  



1it [00:03,  3.26s/it]

Elapsed Time: 3.2606 seconds, image 18degr_singleTrack__2023_12_18__15_49_26_471(190)crop.tif, 321 images exported 
** Processing image : 18degr_singleTrack__2023_12_18__15_49_26_471(200)crop.tif
     -threshold_value: 319.5
Intensity Norm  from (0 , 1216) to  (0, 65535)  



2it [00:08,  4.25s/it]

Elapsed Time: 5.2339 seconds, image 18degr_singleTrack__2023_12_18__15_49_26_471(200)crop.tif, 744 images exported 


# Generate training data from image planes

In [6]:
# Create folder output 
train_data_path = os.path.join(dirOut, 'train_data/')
createFolder(train_data_path)

In [7]:
# Initialize arrays

xy = []
xy_lr = []
xz = []

# Loop over lateral images
file_list_xy = os.listdir(xy_data)
for i in tqdm(range(0, len(file_list_xy), xy_interval)):
    xy_img = tifffile.imread(xy_data + str(i + 1) + '.tif')
    xy_lr_img = tifffile.imread(xy_lr_data + str(i + 1) + '.tif')
    L0 = min(xy_img.shape[0], xy_lr_img.shape[0])
    L1 = min(xy_img.shape[1], xy_lr_img.shape[1])
    for m in range(0, L0 - patch_size + 1, stride):
        for n in range(0, L1 - patch_size + 1, stride):
            crop_xy    =    xy_img[m:m + patch_size, n:n + patch_size]
            crop_xy_lr = xy_lr_img[m:m + patch_size, n:n + patch_size]
            
            if np.max(crop_xy) >= signal_intensity_threshold:
                xy.append(crop_xy)
                xy_lr.append(crop_xy_lr)
                if add_flipped_pathes:
                    xy.append(cv2.flip(crop_xy, 1))
                    xy_lr.append(cv2.flip(crop_xy_lr, 1))                    

# Loop over axial images   
file_list_xz = os.listdir(xz_data)
for i in tqdm(range(0, len(file_list_xz), xz_interval)):
    xz_img = tifffile.imread(xz_data + str(i + 1) + '.tif')
    for m in range(0, xz_img.shape[0] - patch_size + 1, stride):
        for n in range(0, xz_img.shape[1] - patch_size + 1, stride):
            crop_xz = xz_img[m:m + patch_size, n:n + patch_size]

            if np.max(crop_xz) >= signal_intensity_threshold:
                xz.append(crop_xz)
                if add_flipped_pathes:
                    xz.append(cv2.flip(crop_xz,1))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2375/2375 [00:16<00:00, 146.75it/s]


In [8]:
print(len(xy))
print(len(xy_lr))
print(len(xz))

9288
9288
3856


In [9]:
# Convert to arays and save

xy = np.array(xy, dtype=np.float32)
xy_lr = np.array(xy_lr, dtype=np.float32)
xz = np.array(xz, dtype=np.float32)
print(xy.shape, xy_lr.shape, xz.shape)

np.savez(os.path.join(train_data_path, 'train_data.npz'), xy=xy, xy_lr=xy_lr, xz=xz)

(9288, 64, 64) (9288, 64, 64) (3856, 64, 64)


In [10]:
#Save tiff to double check

import tifffile as tiff
tiff.imwrite(os.path.join(train_data_path,'xy.tif'), xy)
tiff.imwrite(os.path.join(train_data_path,'xy_lr.tif'), xy_lr)
tiff.imwrite(os.path.join(train_data_path,'xz.tif'), xz)